## github fork repository crawling using selenium


In [1]:
from selenium import webdriver
import numpy as np
import time
import pandas as pd
import datetime
import requests
import json

In [2]:
def send_slack(channel, username, icon_emoji, message):
    # base_url = "" : github incoming webhook key
    payload = {
       "channel": channel,
       "username": username,
       "icon_emoji": icon_emoji,
       "text": message
       }
    response = requests.post(base_url, data=json.dumps(payload))
    print(response.content)

def slack(function):
    def wrapper(*args, **kwargs):
        name = function.__name__
        start_time = time.time()
        current_time = str(datetime.datetime.now())
        send_slack("project", "databot", ":ghost:", "작업을 실행합니다 - {time}".format(time=current_time))
        
        try:
            result = function(*args, **kwargs)
            current_time = str(datetime.datetime.now())
            end_time = time.time()
            send_slack("project", "databot", ":ghost:", "작업이 끝났습니다 - 걸린시간은 {time}s 입니다.".format(time=int(end_time-start_time)))
            
        except:
            send_slack("project", "databot", ":ghost:", "오류가 났어요 다시 봐주세요.")
            
        current_time = str(datetime.datetime.now())
        end_time = time.time()
        send_slack("dss", "databot", ":ghost:", "작업이 끝났습니다 - 걸린시간은 {time}s 입니다.".format(time=int(end_time-start_time)))
       
    return wrapper

In [6]:
url = "https://github.com/search?utf8=%E2%9C%93&q=language%3APython+location%3A%22republic+korea%22"

In [7]:
driver = webdriver.Firefox()
driver.get(url)
time.sleep(2)

WebDriverException: Message: Can't load the profile. Profile Dir: C:\Users\Bsy\AppData\Local\Temp\tmpn7iu463n If you specified a log_file in the FirefoxBinary constructor, check it for details.


In [ ]:
# 코드가 매우 더럽습니다..하하.... 코딩 공부한지 얼마 안되서..  def에 적응하려고 하고 있습니다

In [77]:
start_time = time.time()
current_time = str(datetime.datetime.now())
send_slack("project", "databot", ":ghost:", "작업을 실행합니다 - {time}".format(time=current_time))

try:
    user_n = driver.find_elements_by_css_selector("div.user-list-info")
    user_df = pd.DataFrame(columns=["Full_name","User_name","Country","Mail_address","Webpage","Join_date","Followers",\
                                        "Starred","Following","Contributions"], )
    repo_df = pd.DataFrame(columns=["ID", "master","repo_name", "Watch", "Star", "Fork", "commit", "branch", "releases", "contributor", \
                                    "repo_last_commit","lan_1", "per_1", "lan_2", "per_2", "lan_3", "per_3", "lan_4", "per_4", "lan_5","pre_5", \
                                    "lan_6","pre_6", "lan_7","pre_7","readme_text"])
    for b in range(20):
        for a in range(len(user_n)): 
            driver.find_elements_by_css_selector("div.user-list-info")[a].find_element_by_css_selector("a").click()
            time.sleep(1.4)

            user_dict = {
            "Full_name": driver.find_element_by_css_selector("div.vcard-fullname").text,
            "User_name": driver.find_element_by_css_selector("div.vcard-username").text,
            "Country": driver.find_element_by_css_selector("li[aria-label='Home location").text if "Home location" in  driver.page_source else "",
            "Mail_address": driver.find_element_by_css_selector("li[aria-label='Email").text if "Email" in  driver.page_source else "",
            "Webpage": driver.find_element_by_css_selector("li[aria-label='Blog or website").text if "Blog or website" in  driver.page_source else "",
            "Join_date": driver.find_element_by_css_selector("li[aria-label='Member since']").text,
            "Followers": driver.find_element_by_css_selector("div.vcard-stats").text.split("\n")[0],
            "Starred": driver.find_element_by_css_selector("div.vcard-stats").text.split("\n")[2],
            "Following": driver.find_element_by_css_selector("div.vcard-stats").text.split("\n")[4],
            "Contributions": driver.find_elements_by_css_selector("div.boxed-group.flush")[-1].find_element_by_css_selector("h3").text.split(" ")[0],   
            }


            user_df.loc[len(user_df)] = user_dict


            Table = driver.find_elements_by_css_selector("div.column.three-fourths")
            Table[0].find_element_by_css_selector("svg.octicon.octicon-repo").click()
            time.sleep(1.5)

            Contents = driver.find_elements_by_css_selector("div.repo-list-item.public.fork")
            for k in range(len(Contents)):
                Contents = driver.find_elements_by_css_selector("div.repo-list-item.public.fork")
                ID = driver.find_element_by_css_selector("div.vcard-username").text    #ID

                if len(Contents[k].find_elements_by_css_selector("div.repo-list-stats")[0].text.split(" ")) == 3:
                    Contents[k].find_element_by_css_selector("p.repo-list-info").find_element_by_css_selector("a").click() # fork의 원본으로 클릭
                    if "blankslate blankslate-narrow text-left" in driver.page_source:
                        driver.execute_script("window.history.go(-1)")
                    else:
    
                        master = driver.find_element_by_css_selector("span.author").text
                        repo_name = driver.find_element_by_css_selector("h1.public").find_elements_by_css_selector("a")[-1].text    # repository

                        social_list = [
                            driver.find_elements_by_css_selector("a.social-count")[0].text,    #Watch
                            driver.find_elements_by_css_selector("a.social-count")[1].text,    #Star
                            driver.find_elements_by_css_selector("a.social-count")[2].text,    #Fork
                            ]

                        repo_content = driver.find_element_by_css_selector("div.repository-content")
                        repo_list = [
                            repo_content.find_elements_by_css_selector("span.num.text-emphasized")[0].text,    #commit
                            repo_content.find_elements_by_css_selector("span.num.text-emphasized")[1].text,    #branch
                            repo_content.find_elements_by_css_selector("span.num.text-emphasized")[2].text,    #releases
                            repo_content.find_elements_by_css_selector("span.num.text-emphasized")[3].text,    #contributor
                            ]    
                        repo_last_commit = driver.find_element_by_css_selector("div.commit-tease.js-details-container").\
                        find_element_by_css_selector("relative-time").text if "relative-time" in  driver.page_source else ""

                        time.sleep(1.4)
                        
                        # language list part
                        try:
                            driver.find_element_by_css_selector("div.repository-lang-stats-graph.js-toggle-lang-stats").click()
                            language_n = driver.find_element_by_css_selector("ol.repository-lang-stats-numbers").find_elements_by_css_selector("li")
                            language_list = [] 
                            for i in range(len(language_n)):
                                language_list.append(language_n[i].text.split(" ")[0])
                                language_list.append(float(language_n[i].text.split(" ")[1][:-1]))
                            if len(language_n) != 7:
                                for i in range(7 - len(language_n)):
                                    language_list.append("")
                                    language_list.append(0.0)
                        except:
                            language_list = ["", 0.0] * 7

                        if 'markdown-body' in driver.page_source:
                            readme_text = repo_content.find_element_by_css_selector("article.markdown-body.entry-content").text.split("\n")[1:]
                        else:
                            readme_text = ""

                        repo_df.loc[len(repo_df)] = [ID, master, repo_name, *social_list, *repo_list, repo_last_commit, *language_list, readme_text]
                        driver.execute_script("window.history.go(-1)")
                        time.sleep(1.5)

            driver.execute_script("window.history.go(-2)")
            time.sleep(1.5)


        driver.find_element_by_css_selector("a.next_page").click()
        time.sleep(2)
    
except:
    send_slack("project", "databot", ":ghost:", "오류가 났으니 확인바랍니다")
        
        
current_time = str(datetime.datetime.now())
end_time = time.time()
send_slack("project", "databot", ":ghost:", "작업이 끝났습니다 - 걸린시간은 {time}s 입니다.".format(time=int(end_time-start_time)))

b'ok'
b'ok'


In [13]:
# 특정 ID의 팔로워들 긁기 ( certain user's followe crawling)
driver = webdriver.Firefox()
url = "https://github.com/dobestan/followers?page=2"
driver.get(url)
time.sleep(2)


start_time = time.time()
current_time = str(datetime.datetime.now())
send_slack("project", "databot", ":ghost:", "작업을 실행합니다 - {time}".format(time=current_time))

try:
    user_n = driver.find_elements_by_css_selector("li.follow-list-item.left.border-bottom")
#     user_df = pd.DataFrame(columns=["Full_name","User_name","Country","Mail_address","Webpage","Join_date","Followers",\
#                                         "Starred","Following","Contributions"], )
#     repo_df = pd.DataFrame(columns=["ID", "master","repo_name", "Watch", "Star", "Fork", "commit", "branch", "releases", "contributor", \
#                                     "repo_last_commit","lan_1", "per_1", "lan_2", "per_2", "lan_3", "per_3", "lan_4", "per_4", "lan_5","pre_5", \
#                                     "lan_6","pre_6", "lan_7","pre_7","readme_text"])

    for a in range(len(user_n)): # 이부분이 몇번째부터 크롤링할지임
        driver.find_elements_by_css_selector("div.follower-list-align-top.d-inline-block.ml-3")[a].find_element_by_css_selector("a").click()
        time.sleep(2)

#         user_dict = {
#         "Full_name": driver.find_element_by_css_selector("div.vcard-fullname").text,
#         "User_name": driver.find_element_by_css_selector("div.vcard-username").text,
#         "Country": driver.find_element_by_css_selector("li[aria-label='Home location").text if "Home location" in  driver.page_source else "",
#         "Mail_address": driver.find_element_by_css_selector("li[aria-label='Email").text if "Email" in  driver.page_source else "",
#         "Webpage": driver.find_element_by_css_selector("li[aria-label='Blog or website").text if "Blog or website" in  driver.page_source else "",
#         "Join_date": driver.find_element_by_css_selector("li[aria-label='Member since']").text,
#         "Followers": driver.find_element_by_css_selector("div.vcard-stats").text.split("\n")[0],
#         "Starred": driver.find_element_by_css_selector("div.vcard-stats").text.split("\n")[2],
#         "Following": driver.find_element_by_css_selector("div.vcard-stats").text.split("\n")[4],
#         "Contributions": driver.find_elements_by_css_selector("div.boxed-group.flush")[-1].find_element_by_css_selector("h3").text.split(" ")[0],   
#         }


#         user_df.loc[len(user_df)] = user_dict


        Table = driver.find_elements_by_css_selector("div.column.three-fourths")
        Table[0].find_element_by_css_selector("svg.octicon.octicon-repo").click()
        time.sleep(2)



        Contents = driver.find_elements_by_css_selector("div.repo-list-item.public.fork")
        for k in range(len(Contents)):
            Contents = driver.find_elements_by_css_selector("div.repo-list-item.public.fork")
            ID = driver.find_element_by_css_selector("div.vcard-username").text    #ID

            if len(Contents[k].find_elements_by_css_selector("div.repo-list-stats")[0].text.split(" ")) == 3:
                Contents[k].find_element_by_css_selector("p.repo-list-info").find_element_by_css_selector("a").click() # fork의 원본으로 클릭

                master = driver.find_element_by_css_selector("span.author").text
                repo_name = driver.find_element_by_css_selector("h1.public").find_elements_by_css_selector("a")[-1].text    # repository

                social_list = [
                    driver.find_elements_by_css_selector("a.social-count")[0].text,    #Watch
                    driver.find_elements_by_css_selector("a.social-count")[1].text,    #Star
                    driver.find_elements_by_css_selector("a.social-count")[2].text,    #Fork
                    ]

                repo_content = driver.find_element_by_css_selector("div.repository-content")
                repo_list = [
                    repo_content.find_elements_by_css_selector("span.num.text-emphasized")[0].text,    #commit
                    repo_content.find_elements_by_css_selector("span.num.text-emphasized")[1].text,    #branch
                    repo_content.find_elements_by_css_selector("span.num.text-emphasized")[2].text,    #releases
                    repo_content.find_elements_by_css_selector("span.num.text-emphasized")[3].text,    #contributor
                    ]    
                repo_last_commit = driver.find_element_by_css_selector("div.commit-tease.js-details-container").\
                find_element_by_css_selector("relative-time").text if "relative-time" in  driver.page_source else ""

                time.sleep(2)
                # language list part
                try:
                    driver.find_element_by_css_selector("div.repository-lang-stats-graph.js-toggle-lang-stats").click()
                    language_n = driver.find_element_by_css_selector("ol.repository-lang-stats-numbers").find_elements_by_css_selector("li")
                    language_list = [] 
                    for i in range(len(language_n)):
                        language_list.append(language_n[i].text.split(" ")[0])
                        language_list.append(float(language_n[i].text.split(" ")[1][:-1]))
                    if len(language_n) != 7:
                        for i in range(7 - len(language_n)):
                            language_list.append("")
                            language_list.append(0.0)
                except:
                    language_list = ["", 0.0] * 7

                if 'markdown-body' in driver.page_source:
                    readme_text = repo_content.find_element_by_css_selector("article.markdown-body.entry-content").text.split("\n")[1:]
                else:
                    readme_text = ""

#                 if k % 5 == 0:
#                     print(k)

                repo_df.loc[len(repo_df)] = [ID, master, repo_name, *social_list, *repo_list, repo_last_commit, *language_list, readme_text]
                driver.execute_script("window.history.go(-1)")
                time.sleep(2)

        driver.execute_script("window.history.go(-2)")
        time.sleep(2)

except:
    send_slack("project", "databot", ":ghost:", "오류가 났으니 확인바랍니다")
        
        
current_time = str(datetime.datetime.now())
end_time = time.time()
send_slack("project", "databot", ":ghost:", "작업이 끝났습니다 - 걸린시간은 {time}s 입니다.".format(time=int(end_time-start_time)))

b'ok'
b'ok'
